# Descripción del proyecto

## Objetivo

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

Condiciones:
- Solo se debe usar la regresión lineal para el entrenamiento del modelo.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

In [2]:
#Se importan las librerías que se estarán utilizando para el proyecto

import pandas as pd
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
#Se cargan los datasets que se estarán utilizando para el proyecto

gdata_0 = pd.read_csv('/datasets/geo_data_0.csv')
gdata_1 = pd.read_csv('/datasets/geo_data_1.csv')
gdata_2 = pd.read_csv('/datasets/geo_data_2.csv')

pd.options.mode.chained_assignment = None

Se examina la información de cada uno de los datasets para conocer al tipo de información que nos presentamos, y posteriormente saber como tratarla, en caso de ser necesario.

In [3]:
gdata_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
gdata_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
gdata_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
gdata_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [7]:
gdata_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
gdata_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## Entrenamiento del modelo

In [9]:
#Se remueven las columnas que se considera que no son prescindibles para realizan las predicciones

gdata_0 = gdata_0.drop(columns = ['id'])
gdata_0.sample(10)

,f0,f1,f2,product
61610,1.130386,-0.441906,-2.257733,56.572799
56485,-0.848382,0.351530,1.923099,76.104538
90647,1.353919,-0.373898,0.967246,87.374560
84197,0.895511,-0.552529,1.863586,119.849452
81393,1.157257,-0.453746,-2.338338,90.228425
34667,-0.749415,0.816496,-0.323793,87.776423
27444,0.173758,-0.433911,1.514347,73.069829
88575,0.819594,0.676734,1.662984,42.385929
72938,0.071754,1.024392,-3.165249,2.901772
15225,1.243883,-0.387798,9.947105,151.576121


In [10]:
features0 = gdata_0.drop(['product'], axis=1) #Se extrae la columna objetivo de las características
target0 = gdata_0['product'] #Se asigna la columna objetivo

In [11]:
#se segmenta el 25% de los datos para hacer el conjunto de validación

features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, 
    target0, 
    test_size=0.25, 
    random_state=12345
)

In [12]:
#Escalado de características - Estandarización

numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train0[numeric])

#Transformar los datos
features_train0[numeric] = scaler.transform(features_train0[numeric])
features_valid0[numeric] = scaler.transform(features_valid0[numeric])

features_train0.head()

,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567
62041,0.260460,0.825069,-0.204865
70185,-1.837105,0.010321,-0.147634
82230,-1.299243,0.987558,1.273181


In [13]:
features_train0[numeric].describe().apply(round)

,f0,f1,f2
count,75000.0,75000.0,75000.0
mean,0.0,-0.0,0.0
std,1.0,1.0,1.0
min,-2.0,-2.0,-4.0
25%,-1.0,-1.0,-1.0
50%,0.0,-0.0,0.0
75%,1.0,1.0,1.0
max,2.0,2.0,4.0


In [14]:
features_valid0[numeric].describe().apply(round)

,f0,f1,f2
count,25000.0,25000.0,25000.0
mean,0.0,0.0,-0.0
std,1.0,1.0,1.0
min,-2.0,-2.0,-4.0
25%,-1.0,-1.0,-1.0
50%,0.0,0.0,-0.0
75%,1.0,1.0,1.0
max,2.0,2.0,4.0


In [15]:
features_train0.sample(5)

,f0,f1,f2
71592,-0.649330,-0.200508,-0.085337
83291,0.967111,-1.344335,-0.599326
6971,0.172448,0.773332,-0.246700
58871,-0.541633,0.537776,-0.303746
17408,-0.108631,1.257758,-2.550992


In [16]:
model = LinearRegression() # inicializa el constructor de modelos
model.fit(features_train0, target_train0) # entrena el modelo en el conjunto de entrenamiento
predictions_valid0 = model.predict(features_valid0) # obtén las predicciones del modelo en el conjunto de validación

mean0 = predictions_valid0.mean()
target_valid0 = target_valid0.reset_index(drop=True)
predictions_valid0 = pd.Series(predictions_valid0)

result0 = mean_squared_error(target_valid0, predictions_valid0)**0.5 # calcula la RECM en el conjunto de validación
print("RMSE del modelo de regresión lineal en el conjunto de validación:", result0)
print("Volúmen medio de reservas:", mean0)

RMSE del modelo de regresión lineal en el conjunto de validación: 37.5794217150813
Volúmen medio de reservas: 92.59256778438035


In [17]:
#Se realiza la Prueba de cordura

predicted_valid0 = pd.Series(target_train0.mean(), index=target_valid0.index)
mse0 = mean_squared_error(target_valid0, predicted_valid0)
    
print('Prueba de cordura')
print('MSE =', mse0)
print('RMSE =', mse0 ** 0.5)

Prueba de cordura
MSE = 1961.5678757223516
RMSE = 44.289591053907365


In [18]:
#Pasando a funciones pasos previos para aplicarlos a las regiones faltantes

def modelo (df, features, target):
    
    # segmenta el 25% de los datos para hacer el conjunto de validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, 
        target, 
        test_size=0.25, 
        random_state=12345
    )
    
    #Escalado de características - Estandarización
    numeric = ['f0', 'f1', 'f2']

    scaler = StandardScaler()
    scaler.fit(features_train[numeric])

    #Transformar los datos
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])

    features_train.head()
    
    model = LinearRegression() 
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid) 

    mean = predictions_valid.mean()
    target_valid = target_valid.reset_index(drop=True)
    
    predictions_valid = pd.Series(predictions_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5 
    print("RMSE del modelo de regresión lineal en el conjunto de validación:", result)
    print("Volúmen medio de reservas:", mean)
    print()
    
    #Se realiza la Prueba de cordura
    predicted_valid = pd.Series(target_train.mean(), index=target_valid.index)
    mse = mean_squared_error(target_valid, predicted_valid)

    print('Prueba de cordura')
    print('MSE =', mse)
    print('RMSE =', mse ** 0.5)
    
    return target_valid, result, mean, predictions_valid

In [19]:
#Se remueven las columnas que se considera que no son prescindibles para realizan las predicciones

gdata_1 = gdata_1.drop(columns = ['id'])
gdata_1.sample(10)

,f0,f1,f2,product
21110,-3.280062,-2.686901,4.004253,110.992147
9393,11.403736,-3.116396,-0.007355,0.000000
43887,3.039728,-5.296816,5.001500,134.766305
8247,13.502119,0.463354,3.003647,80.859783
78316,0.422409,-5.062173,0.994122,30.132364
32135,-0.006197,-5.048122,2.000389,57.085625
95076,11.579775,-2.869554,1.003051,26.953261
57302,10.153843,0.348896,0.004435,0.000000
88623,-11.240220,-10.329115,2.998659,84.038886
62565,8.480175,-1.278552,-0.012756,0.000000


In [20]:
features1 = gdata_1.drop(['product'], axis=1) #Se extrae la columna objetivo de las características
target1 = gdata_1['product'] #Se asigna la columna objetivo

In [21]:
#Se entrena el modeto de la región 1

target_valid1, result1, mean1, predictions_valid1 = modelo(gdata_1, features1, target1)

RMSE del modelo de regresión lineal en el conjunto de validación: 0.893099286775617
Volúmen medio de reservas: 68.728546895446

Prueba de cordura
MSE = 2117.9734309299147
RMSE = 46.02144533725462


In [22]:
#Se remueven las columnas que se considera que no son prescindibles para realizan las predicciones

gdata_2 = gdata_2.drop(columns = ['id'])
gdata_2.sample(10)

,f0,f1,f2,product
56979,0.148639,0.954589,3.717128,72.293743
3063,-2.109929,0.242462,-0.070448,111.749096
65456,-0.016221,-1.930835,2.921053,47.175453
33536,-1.702089,0.125917,-1.180326,40.470811
64697,-0.011876,0.179763,-3.067359,99.784953
68907,-0.382081,2.997638,5.632393,80.355823
84964,1.087359,0.601750,8.023379,116.805040
96124,-1.172546,3.345244,-0.058521,128.944565
1243,-2.527621,-1.494803,-2.085589,124.659301
79773,-1.321886,-1.227473,0.035542,146.590598


In [23]:
features2 = gdata_2.drop(['product'], axis=1) #Se extrae la columna objetivo de las características
target2 = gdata_2['product'] #Se asigna la columna objetivo

In [24]:
#Se entrena el modelo para la región 2

target_valid2, result2, mean2, predictions_valid2 = modelo(gdata_2, features2, target2)

RMSE del modelo de regresión lineal en el conjunto de validación: 40.02970873393434
Volúmen medio de reservas: 94.96504596800489

Prueba de cordura
MSE = 2016.2210072435087
RMSE = 44.90234968510566


Como podemos visualizar, al analizar el volúmen medio de reservas de cada una de las regiones, tenemos que en la región 2 el volúmen promedio es menor al esperado para evitar pérdida, sin embargo es el volúmen más alto de las tres regiones, por lo que hasta el momento es el que se mira más apto para el desarrollo de los pozos. Sin embargo, aún falta realizar análisis más profundo, para tomar una decisión y realizar una recomendación.

## Calcular ganancias

Dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). Compara esta cantidad con la cantidad media de reservas en cada región.
Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).


- inversión = 100 mill
- pozos = 200 pzs
- 1 pozo = 500,000 usd (111.1 u)
- 1 barril = 4.5 usd
- 1 unidad = 1,000 barriles
- 1 unidad = 4,500 usd

In [25]:
#función para calcular las ganancias

def income_calc (predictions_valid, target_valid):
    predictions_valid=predictions_valid.reset_index(drop=True)
    target_valid=target_valid.reset_index(drop= True)
    best = predictions_valid.sort_values(ascending=False).head(200)
    best = target_valid.iloc[best.index]
    best = pd.DataFrame(best, columns=['product'])
    
    best['income'] = best['product'].multiply(4500)
    total_income = best['income'].sum()
    
    return best, total_income 

In [26]:
best0, total_income0 = income_calc(predictions_valid0, target_valid0)
print('Ganancia por el top 200 de pozos en región 0: ',total_income0 - 100000000)

best1, total_income1 = income_calc(predictions_valid1, target_valid1)
print('Ganancia por el top 200 de pozos en región 1: ',total_income1 - 100000000)

best2, total_income2 = income_calc(predictions_valid2, target_valid2)
print('Ganancia por el top 200 de pozos en región 2: ',total_income2 - 100000000)

Ganancia por el top 200 de pozos en región 0:  33208260.43139851
Ganancia por el top 200 de pozos en región 1:  24150866.966815084
Ganancia por el top 200 de pozos en región 2:  27103499.63599831


Al visualizar los resultados, podemos observar que de todas las regiones, la región 2 es la que continúa en el top como el mejor lugar, ya que en esta ocasión muestra que es la opción que arroja la mayor ganancia potencial, dando un aproximado de 69.4 millones de dólares.

En este punto, se continuaría teniendo como top la región 2 para realizar el desarrollo de los pozos ya que hasta el momento es la más idónea tanto para realizar la inversión, como para obtener las ganancias y un retorno de inversión de manera más acelerada.

## Calcular riesgos y ganancias

In [27]:
#Función bootstrapping para calcular el beneficio promerio, riesgos de pérdidas e intervalo de confianza a un 95%, para para
#cada una de las regiones

def bootstrapping(target_valid, predictions_valid):
    
    values = []
    state = np.random.RandomState(12345)
    
    for i in range(1000):
        
        target_subsample = target_valid.sample(n=500, random_state=state, replace=True)
        probs_subsample = predictions_valid.iloc[target_subsample.index]
        
        _,total_income = income_calc(probs_subsample, target_subsample)
        revenue = total_income - 100000000
        values.append(revenue)

    values = pd.Series(values)

    mean = values.mean()
    intervalo_confianza = np.percentile(values, [2.5, 97.5])
    
    probability_of_lose = np.mean(values < 0)
    risk = probability_of_lose * 100
    
    print(f'Beneficio promedio: ${mean:.2f}')
    print(f'Intervalo de confianza del 95%: (${intervalo_confianza[0]:.2f}, ${intervalo_confianza[1]:.2f})')
    print(f'Riesgo de pérdidas: {risk:.2f}%')

In [28]:
bootstrapping(target_valid0, predictions_valid0)

Beneficio promedio: $3961649.85
Intervalo de confianza del 95%: ($-1112155.46, $9097669.42)
Riesgo de pérdidas: 6.90%


In [29]:
bootstrapping(target_valid1, predictions_valid1)

Beneficio promedio: $4560451.06
Intervalo de confianza del 95%: ($338205.09, $8522894.54)
Riesgo de pérdidas: 1.50%


In [30]:
bootstrapping(target_valid2, predictions_valid2)

Beneficio promedio: $4044038.67
Intervalo de confianza del 95%: ($-1633504.13, $9503595.75)
Riesgo de pérdidas: 7.60%


Al realizar el cálculo de riesgos y ganancias en las tres regiones, podemos observar en la región 1 que, aunque su intervalo de confianza no es el más alto en comparación con las otras dos regiones, si lo es en beneficio promedio y riesgos de pérdida, siendo este último resultado (1.50% pérdida) el único que encaja en los parámetros estipulados en el proyecto. Por lo tanto, en este caso la recomendación sería la región 1.

## Conclusión

Al realizar el cálculo de beneficio promedio, intervalo de confianza y riesgos, podemos observar que la información y recomendación da un giro, esto es porque al analizar los resultados obtenidos en el último bloque, pudimos ver que la que mayor beneficio promedio y menor riesgo representa es la región 1, por lo tanto la recomendación que se proporcionaría es realizar el desarrollo de los 200 pozos es la región 1.